In [1]:
import os
import tweepy as tw
import pandas as pd
from textblob import TextBlob

In [2]:
# Go to http://apps.twitter.com and create an app.
# The consumer key and secret will be generated for you after
consumer_key="rlCkqsZTQVtPbn1eNcp8W4nRB"
consumer_secret="Ek4XgMEjz4yVxYnwtM2a2J1yodMjXY5mwcRJJnJDvcGh3FODqC"

# After the step above, you will be redirected to your app's page.
# Create an access token under the the "Your access token" section
access_token="122083690-XdJNW0LsR9TzZEGjAojvCqYW3LgZS2ZNZyH1mgWU"
access_token_secret="moCTCORNupEB16iJqTXScJmUGjbPmJHNfh7FBuXsvtHRA"

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
place = api.geo_search(query="MN", granularity="admin")
place_id = place[0].id

In [5]:
dict_topic = {"Social Security":["social security","socialsecuritylaws","social security laws","socialsecurityissues","social security issues","socialsecuritypolicy","social security policy","retirement","retirementpolicy","retirement policy","senior citizen","senior citizen laws","seniorcitizenlaws","employee benefits","retirement plan","tax exemption","taxexemption"],
              "Immigration":["immigration","immigration laws","immigrationlaws","immigration policy","immigrationpolicy","national immigration","immigrant","immigration new","mexican immigration","for immigration","forimmigration","against immigration","againstimmigration","anti-immigration","illegal mexican","immigrations","legal immigration","legalimmigration","against immigration","immigrants rights","immigrantsrights","american immigration","us immigration law","usimmigrationlaw","immigration mexico","immigration policies","immigration studies","immigrant population","center for immigration","immigrant rights","immigrant workers","mexican immigrants","immigration requirements","immigrant statistics","immigration history","immigration coalition","immigration illegal","immigration quotas","immigration control","immigration statistics","immigration problems","refugee rights","immigration and nationality","illegal mexicans","us immigration","pro immigration","undocumented workers","immigration policy","recent immigration","immigration green card","immigration issues","mexico immigration","american immigration reform","current immigration","immigration numbers","us immigration laws","immigration rights","u.s. immigration","louvre museum","immigrants","border control","u.s immigration","us immigrant","immigration us","immigration reform and control act","immigration legal","national immigration forum","new immigrant","visas","green cards","illegal mexican immigrants","mexifornia","america immigration","immigration law center","border security","fair immigration","federation for american immigration reform","nationality act","visa immigration","immigration rules","immigration and nationality act","hispanic immigration","immigration in america","illegal immigration","migration policy","new immigration laws","immigrant worker","american border","us immigrants","immigrant labor","immigration act"],
              "Medicare":["medicare insurance","medicare part d plan","medicare plans","help with medicare","medigap plan","medicare advantage","medicare.","medicare supplement","medicare help","medicare part c","medicare,gov","medicare.gove","medicare coverage","medicare cost","medicare advantage plans","medicare enrollment","cost of medicare","people healthcare","healthcare","healthcare for people","healthcareforpeople","healthcare policy","healthcarepolicy","healthcare policies","healthcare issues","healthcareissue","healthcare problems","issues in healthcare","government healthcare"],
              "Crime":["crime","crime policy","crimepolicy","crime laws","crimelaws","crime issues","crimeissues","criminal","criminal act","criminallaws","policelaws","police laws","criminal justice"],
              "Unemployment":["unemployment","illeterate unemployment","unemployment issues","unemployment check","unempploymentissues","unemployment policy","unemployment online","unemploy","unemployed","unemployment laws","unemploymentlaws","justice for unemployment"]
             }

In [6]:
only_tweets = []
topic_list = []

for key in dict_topic.keys():
    keyword_list = dict_topic[key]

    for word in keyword_list:
        new_search = str(word) + " -filter:retweets" 
        tweets = tw.Cursor(api.search, 
                               q=new_search+ ' place:%s' % place_id,
                               lang="en",
                               since="2019-11-08").items(1000)
        only_tweets.append(tweets)
        topic_list.append(key)
        

In [7]:
len(only_tweets), len(topic_list)

(162, 162)

In [8]:
# Getting the tweets into an array which would be then be loaded to dataframe
full_arr= []
topic_arr = []
i=0
for items in only_tweets:
    cur_topic = topic_list[i]
    # print(cur_topic)
    #print(items)
    i = i+1
    for tweet in items:
        #print(tweet)
        arr = [tweet.user.name, tweet.place.name,tweet.place.full_name, tweet.user.followers_count, tweet.created_at, tweet.retweet_count, tweet.text]
        #print(arr)
        full_arr.append(arr)
        topic_arr.append(cur_topic)

In [9]:
len(full_arr), len(topic_arr)

(428, 428)

In [60]:
final_dataframe = pd.DataFrame(full_arr,columns = ['user', "place","place_full_name", "follower count", "created at","retweets", "text"])
final_dataframe['sentiment'] = round(final_dataframe.text.apply(lambda x: TextBlob(x).sentiment[0]),2)

In [61]:
final_dataframe['topic'] = pd.Series(topic_arr)

In [62]:
def sentiment_bucket(value):
    sentiment = ""
    if value > 0.02:
        sentiment = "Positive"
    elif value < -0.02:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return sentiment

final_dataframe['SentimentBucket'] = final_dataframe['sentiment'].apply(sentiment_bucket)

def influencer_level(count):
    level = ""
    if count > 1000000:
        level = "Key"
    elif count > 100000:
        level = "Big"
    elif count > 10000:
        level = "Medium"
    elif count > 5000:
        level = "Local"
    else:
        level = "Not"
    return level

final_dataframe['influencerLevel'] = final_dataframe["follower count"].apply(influencer_level)


In [63]:
final_dataframe.head(5)

,user,place,place_full_name,follower count,created at,retweets,text,sentiment,topic,SentimentBucket,influencerLevel
0,Jean Mari Burrington,Minnesota,"Minnesota, USA",39,2019-12-09 00:46:33,0,@BeltwayPanda @SenKamalaHarris @SpryGuy @FLOTU...,0.20,Immigration,Positive,Not
1,TMJ-MSP Legal Jobs,Minneapolis,"Minneapolis, MN",327,2019-12-08 17:43:09,0,This #Legal job might be a great fit for you. ...,0.48,Immigration,Positive,Not
2,Stephen Philion,Maple Grove,"Maple Grove, MN",145,2019-12-08 16:15:07,0,Mother of Army officer to be deported after re...,0.00,Immigration,Neutral,Not
3,🇺🇸USMC 0341 MAGA 🇺🇸🎖🇺🇸🇺🇸,Minnesota,"Minnesota, USA",70115,2019-12-08 05:53:50,0,@JohnSmi37563557 Exactly.. anothe 9/11 would r...,0.12,Immigration,Positive,Medium
4,RH Legal Jobs,Minneapolis,"Minneapolis, MN",155,2019-12-07 16:31:43,0,This job might be a great fit for you: IMMIGRA...,0.47,Immigration,Positive,Not


In [64]:
final_dataframe["place"].unique()

array(['Minnesota', 'Minneapolis', 'Maple Grove', 'St Paul', 'Hutchinson',
       'Monticello', 'Hopkins', 'Plymouth', 'Lakeville', 'Eden Prairie',
       'Duluth', 'Moorhead', 'Lino Lakes', 'St Cloud', 'Edina', 'Andover',
       'New Brighton', 'Chisholm', 'Maplewood', 'Minnetonka',
       'West St Paul', 'Winona', 'Bloomington', 'Zimmerman', 'Chanhassen',
       'Shakopee', 'Little Falls', 'Hastings', 'Roseville',
       'Brooklyn Center', 'Elk River', 'Richfield', 'Zumbrota',
       'Mendota Heights', 'Red Wing', 'Brooklyn Park', 'Coon Rapids',
       'Arden Hills', 'Burnsville', 'Prior Lake', 'Aria', 'Apple Valley',
       'White Bear Lake', 'Champlin', 'Crystal', 'Rogers',
       'Golden Valley', 'Brainerd', 'Victoria', 'Mahtomedi', 'Litchfield',
       'Cambridge', 'Austin', 'Melrose', 'Mound', 'Woodbury', 'Eagan',
       'Marshall', 'Saint Paul City Hall', 'Anoka', 'Robbinsdale',
       'Mounds View', 'Inver Grove Heights', 'Forest Lake', 'Fridley',
       'Cottage Grove', 'Minn

In [65]:
import numpy as np
np.mean(final_dataframe.sentiment)

0.05752336448598132

In [66]:
final_dataframe.to_csv("final_dataframe.csv", index = False)

In [67]:
from google.cloud import storage

client = storage.Client()

bucket = client.bucket("msba-trends")
blob = bucket.blob('final_dataframe.csv')
blob.upload_from_filename('final_dataframe.csv')

In [68]:
final_dataframe.groupby(['topic','SentimentBucket']).size()

topic            SentimentBucket
Crime            Negative           65
                 Neutral            54
                 Positive           41
Immigration      Negative           14
                 Neutral            20
                 Positive           40
Medicare         Negative           19
                 Neutral            52
                 Positive           63
Social Security  Negative            9
                 Neutral             9
                 Positive           21
Unemployment     Negative            1
                 Neutral            12
                 Positive            8
dtype: int64